# Modelos probabilísticos de ortografía

## Ortografía: Dada una secuencia de letras de una palabra mal escrita producir una lista ordenada de palabras posibles escritas de manera correcta

### Aplicaciones: Chats, procesamiento de textos, OCR. 

### 80% de errores con el teclado se debe a errores de un solo caracter en una de las siguientes formas:
 - Inserción:          * carro -> carreo*
 - Supresión:        * carro -> caro*
 - Substitución:    * carro -> carto*
 - Transposición: * carro -> caror*

In [1]:
letters    = "abcdefghijklmnopqrstuvwxyz\'"
def edits1(word):
    "All edits that are one edit away from `word`."
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def known(words):
    return set(w for w in words if w in WORDS)

### Vocabulario

In [2]:
import re
from collections import Counter

with open('data/big.txt', 'r') as f:
    corpus = f.read()

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(corpus))

### Para una palabra dada podemos calcular todas las palabras que se encuentras a distancia uno de ella.

In [3]:
word = 'something'
print(edits1(word), end=' '), len(edits1(word)), known(edits1('somthing'))

{'somethzing', 'somethins', 'somethinw', 'somesthing', 'somethking', 'somsething', 'somethuing', 'somethirg', 'somethiug', 'svomething', 'romething', 'bomething', 'somethingq', 'sodething', 'eomething', 'sosmething', 'syomething', 'somevthing', 'soxething', 'somethieng', 'msomething', 'somejthing', 'somfething', 'somethinig', 'somiething', 'solmething', 'somenthing', "som'thing", 'somekhing', 'sjomething', 'somethifng', 'sometfing', 'sometsing', 'homething', 'somdething', 'sometching', 'someching', 'sometqing', 'somedthing', 'sometjhing', 'lomething', 'somethibg', 'sometging', 'somethincg', 'somethinag', 'tsomething', 'somwething', 'somedhing', 'somthing', 'ssmething', 'somethihng', 'someothing', 'somethink', 'sfomething', 'somethinzg', 'somyething', 'psomething', 'sogmething', 'rsomething', 'sowmething', 'soqething', 'jsomething', 'yomething', 'sombthing', 'sompething', 'somethingv', 'somethixg', 'somethinng', 'dsomething', 'somethnng', 'somethihg', 'oomething', 'somethingg', 'somethi

(None, 513, {'something', 'soothing'})

### La gran mayoria de errores están a una distancia menor o igual a 2. Aproximadamente el 98% 

In [25]:
word = 'somemme'
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [26]:
len(edits2(word)), len(known(edits2(word))), known(edits2(word))

(74325, 2, {'someone', 'sometime'})

### Los errores más comunes para los mecanógrafos son los de substitución por letras adyacentes en el teclado (59%).

### Errores cognitivos: Substitución de una sequencia de letras por otra fonéticamente equivalente, 
- *cerrar -> serrar*

### Errores de OCR:
- **Texto**: The quick brown fox jumps over the lazy dog.
- **Reconocido**: The q~ck brown foxjurnps ovcr tb l azy dog.

## Tarea:
-**Reconocimiento de voz** - Dada la pronunciación de una palabra se desea encontrar la palabra verdadera
-**Corrección de ortografía**- Dada una palabra mal escrita encontrar la palabra verdadera


## Inferencia Bayesiana:  
- P(palabra|obsevación) = : $P(w|O)$
- El objetivo es encontrar la palabra más probable dada la observación:
\begin{equation} \text{argmax}_{w\in V} P(w\,|\,O) 
\end{equation}
- Teorema de Bayes:
\begin{equation} P(x\,|\,y) = \frac{P(y\,|\,x)P(x)}{P(y)} \end{equation}
- El problema se transforma en el siguiente:
\begin{equation} \hat{w} =  \text{argmax}_{w\in V} \frac{P(O\,|\,w)P(w)}{P(O)} \end{equation}
- Las probabilidades del lado derecho son por lo general más fáciles de calcular. En este caso $P(w)$ es la frecuencia relativa de la palabra en el corpus. P(O|w) se calcula de acuerdo al modelo en nuestro caso un canal ruidoso. $P(O)$, la probabilidad de una observación es difícil de calcular, la bueno es que no hay que hacerlo.  
\begin{equation}
\hat{w} =  \text{argmax}_{w\in V} \frac{P(O\,|\,w)P(w)}{P(O)} = \text{argmax}_{w\in V} P(O\,|\,w)P(w)
\end{equation} 
- $P(w)$ se llama la distribución previa y $P(O\,|\,w)$ es la verosimilitud de la observación.


## Modelo de canal ruidoso: 
- Se parte de la palabra correcta y se deforma para encontrar la palabra con errores. Con el análisis bayesiano se intenta encontrar la palabra original. 
- Un primer modelo: se asume que la palabra original difiera de la mal escrita en solo una inserción, supresión, substitución o transposición. 
![title](tabla_errores_ortografia.png)

$$\hat{c} = \text{argmax} P(e\,|\,c)P(c)$$

- $P(c)$ se calcula como la frecuencia de la palabra en el corpus. 
$$P(c) = \frac{C(c) + 0.5}{N + 0.5V},$$ Donde $N$ es la cantidad de palabras en el corpus y $V$ es la cantidad de palabras en el vocabulario.
- $P(e\,|\,c)$ depende de muchos factores, por ejemplo de la persona que esta escribiendo. 

- Una forma sencilla se estimar $P(e\,|\,c)$ es utilizando una matriz de confusión para cada tipo de error.  

### P: frecuencia relativa de las palabras

In [13]:
def P(word, N=sum(WORDS.values())):
    return (WORDS[word]+0.5) / (N + 0.5*len(WORDS))

In [14]:
len(WORDS), WORDS.most_common(n=10), P('the')

(32198,
 [('the', 79809),
  ('of', 40024),
  ('and', 38312),
  ('to', 28765),
  ('in', 22023),
  ('a', 21124),
  ('that', 12512),
  ('he', 12401),
  ('was', 11410),
  ('it', 10681)],
 0.07052277844345241)

### Importar base de datos de errores

In [16]:
mispelled_dict = {}
repeat_re = re.compile(r'(.+)\*(\d+)')
with open("data/spell-errors.txt") as f:
    for line in f:
        (key, val) = line.split(':')
        val = re.sub(pattern= r'\n| ',repl='',string=val)
        val = val.split(",")
        val_new = list()
        for word in val:
            word = word.lower()
            if repeat_re.match(word):                
                repeated = repeat_re.findall(word)[0]
                val_new = val_new + [repeated[0]]*int(repeated[1])
            else:
                val_new = val_new + [word]
        mispelled_dict[key.lower()] = val_new

In [17]:
mispelled_dict

{'raining': ['rainning', 'raning'],
 'writings': ['writtings'],
 'disparagingly': ['disparingly'],
 'yellow': ['yello'],
 'four': ['forer',
  'fours',
  'fuore',
  'fore',
  'fore',
  'fore',
  'fore',
  'fore',
  'for',
  'for',
  'for',
  'for'],
 'woods': ['woodes'],
 'hanging': ['haing'],
 'aggression': ['agression'],
 'looking': ['loking',
  'begining',
  'luing',
  'look',
  'look',
  'locking',
  'lucking',
  'louk',
  'looing',
  'lookin',
  'liking'],
 'eligible': ['eligble', 'elegable', 'eligable'],
 'electricity': ['electrisity', 'electricty', 'electricty', 'electrizity'],
 'scold': ['schold', 'skold'],
 'adaptable': ['adabtable'],
 'caned': ['canned', 'cained'],
 'immature': ['imature'],
 "shouldn't": ['shoudln', 'shouldnt'],
 'swivel': ['swival'],
 'appropriation': ['apropriation'],
 'fur': ['furr', 'fer'],
 'stabbed': ['stabed'],
 'southwold': ['suothwode'],
 'disturb': ['distrebe', 'desturb'],
 'recollections': ['reclections', 'recolections'],
 'prize': ['prise', 'prizer

In [56]:
from itertools import zip_longest
def difference_of_words(word_true, word_mistake):
    # returns mistake type, mistake made and intended letters
    one_step_compare = zip_longest(word_true, word_mistake,
                                   word_true[1:], word_mistake[1:],
                                   ' '+word_true, ' '+word_mistake,
                                   fillvalue='')
    for char_true, char_false, char_true_ahead, char_false_ahead, char_true_prev, char_false_prev in one_step_compare:
        if char_true != char_false:
                # transposition
                if (char_true_ahead == char_false) & (char_false_ahead == char_true):
                    return({'type':'transposition', 'error': char_false + char_false_ahead,
                           'true': char_true + char_true_ahead})
                # substitution
                elif (char_true_ahead == char_false_ahead)\
                    &(len(word_true)==len(word_mistake)):
                    return({'type':'substitution', 'error': char_false,'true': char_true})
                # deletion
                elif char_true_ahead == char_false:
                    if char_false != '':
                        return({'type':'deletion', 'error':char_false,
                                'true': char_true + char_true_ahead})
                    else:
                        return({'type':'deletion', 'error':char_false_prev,
                                'true': char_true_prev + char_true})
                # insertion
                elif char_false_ahead == char_true:
                    if char_true != '':
                        return({'type':'insertion', 'error':char_false + char_false_ahead,
                                'true': char_true})
                    else:
                        return({'type':'insertion', 'error':char_false_prev + char_false,
                                'true': char_true_prev})

In [57]:
difference_of_words('thers', 'therss')

{'error': 'ss', 'true': 's', 'type': 'insertion'}

In [58]:
from collections import defaultdict
def get_confusion_matrix(mispelled_dict):
    mistake_count  = defaultdict(int)
    type_count     = defaultdict(int)
    agg_by_mistake = defaultdict(int)
    for key, val in mispelled_dict.items():
        for mispelled_word in val:
            mistake = difference_of_words(key, mispelled_word)
            if mistake:
                type_count[mistake['type']] += 1
                mistake_count[(mistake['error'],mistake['true'])] +=1
                agg_by_mistake[mistake['true']] += 1
    return(type_count, mistake_count, agg_by_mistake)

In [59]:
type_count, mistake_count, agg_by_mistake = get_confusion_matrix(mispelled_dict)

In [60]:
sorted(mistake_count.items(), key=lambda x: x[1], reverse = True)

[(('e', 'i'), 647),
 (('a', 'e'), 583),
 (('', 'ed'), 535),
 (('e', 'a'), 508),
 (('i', 'e'), 488),
 (('', 'ly'), 387),
 (('s', 'c'), 322),
 (('a', 'i'), 305),
 (('l', 'al'), 284),
 (('e', 'ie'), 266),
 (('a', 'o'), 246),
 (('o', 'a'), 240),
 (('s', 'es'), 236),
 (('u', 'ou'), 232),
 (('l', 'el'), 228),
 (('c', 'sc'), 224),
 (('ie', 'e'), 218),
 (('r', 'ar'), 218),
 (('y', 'ly'), 205),
 (('n', 'in'), 196),
 (('e', 'ce'), 186),
 (('d', 'ed'), 185),
 (('e', 'he'), 185),
 (('n', 'm'), 184),
 (('a', 'ia'), 183),
 (('e', 'o'), 178),
 (('r', 'ur'), 176),
 (('t', 'it'), 174),
 (('e', 'me'), 174),
 (('c', 's'), 171),
 (('', 'ng'), 168),
 (('e', 're'), 168),
 (('i', 'a'), 168),
 (('ea', 'a'), 166),
 (('r', 'er'), 165),
 (('a', 'ua'), 163),
 (('a', 'ra'), 160),
 (('t', 'at'), 158),
 (('n', 'en'), 156),
 (('t', 'nt'), 154),
 (('a', 'la'), 147),
 (('i', 'hi'), 144),
 (('ar', 'r'), 139),
 (('es', 's'), 137),
 (('ou', 'u'), 137),
 (('e', 'u'), 136),
 (('ei', 'i'), 135),
 (('o', 'u'), 135),
 (('er', 

In [61]:
def P(word, N=sum(WORDS.values())):
    return (WORDS[word]+0.5) / (N + 0.5*len(WORDS))

def likelihood(typo, correct):
    if agg_by_mistake[typo] != 0:
        return(mistake_count[(typo, correct)]/(agg_by_mistake[typo]))
    else:
        return(1e-5)
def posterior(correct_word, mispelled_words):
    mistake = difference_of_words(correct_word, mispelled_words)
    return(P(correct_word)*likelihood(mistake['error'], mistake['true']))

In [71]:
def candidates(mispelled_word, max_distance = 2):
    if max_distance == 1:
        candidates = known(edits1(mispelled_word)) or [word]
    elif max_distance == 2:
        candidates = known(edits1(mispelled_word)) or known(edits2(mispelled_word)) or [word]
    candidate_dict = dict()
    for candidate in candidates:
        candidate_dict[candidate] = posterior(candidate, mispelled_word)
    normalizing_constant = sum(candidate_dict.values())
    candidate_dict = {k:v/normalizing_constant for k,v in candidate_dict.items()}
    return(candidate_dict)

In [72]:
sorted(candidates("acress").items(), key=lambda x: x[1], reverse = True)

[('acres', 0.5885981563154824),
 ('across', 0.38995802755760417),
 ('caress', 0.008584594114772063),
 ('actress', 0.006520761434728196),
 ('access', 0.006338460577413214)]

In [73]:
def correction(mispelled_word):
    _candidates = candidates(mispelled_word)
    return(max(_candidates, key=_candidates.get))

In [74]:
candidates('proces')

{'prices': 0.24503567098414214,
 'process': 0.7510410410986045,
 'proves': 0.003923287917253451}

### Pruebas

In [79]:
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = 0
    for right, wrong in tests:
        if (wrong not in WORDS)&(right in WORDS):
            n += 1
            w = correction(wrong)
            good += (w == right)
            if w != right:
                unknown += (right not in WORDS)
                if verbose:
                    print('correction({}) => {} ({}); expected {} ({})'
                          .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} at {:.0f} words per second '
          .format(good / n, n, n / dt))

def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]    

In [80]:
spelltest(Testset(open('data/spell-testset1.txt')))

83% of 252 at 48 words per second 


## Factores adicionales que se pueden incluir en el modelo
- Palabras adyacentes
- Errores personalizados del usuario